### MLP 하이퍼파라미터 튜닝 실험 🔍

<table>
  <tr>
    <td style="vertical-align: top; padding-right: 20px; border: none;">
      <p>MNIST 데이터셋을 활용하여 MLP의 주요 하이퍼파라미터 변화가 모델의 성능에 미치는 영향을 분석하는 실험입니다.</p>
      <b>실험:</b>
      <ol>
        <li>베이스라인 설정</li>
        <li>네트워크 구조 (깊이/너비)</li>
        <li>활성화 함수</li>
        <li>최적화 방법 (옵티마이저/학습률)</li>
      </ol>
    </td>
    <td style="border: none;">
      <img src="mnist_sample.png" width="300"><br>
      <em>MNIST Dataset</em>
    </td>
  </tr>
</table>

---